


# Data Processing

Import libraries and dataframe

In [6]:
pip install scikit-learn

In [7]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

url = 'https://drive.google.com/file/d/1-0exaTj91_tdZ8IxIH0esLd1ty0Uyd6I/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df1 = pd.read_csv(url)

dir = '/content/drive/MyDrive/CS180/Project/CS180_Project_cbragunton_jtderez/IMPLEMENTATION'

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS180/Project/CS180_Project_cbragunton_jtderez/IMPLEMENTATION

In [9]:
print(df1.head(10))

   Global_Sales  Critic_Score  Critic_Count  User_Score  User_Count   highest  \
0         82.53          76.0          51.0         8.0       322.0  NA_Sales   
1         35.52          82.0          73.0         8.3       709.0  NA_Sales   
2         32.77          80.0          73.0         8.0       192.0  NA_Sales   
3         29.80          89.0          65.0         8.5       431.0  NA_Sales   
4         28.92          58.0          41.0         6.6       129.0  NA_Sales   
5         28.32          87.0          80.0         8.4       594.0  NA_Sales   
6         23.21          91.0          64.0         8.6       464.0  NA_Sales   
7         22.70          80.0          63.0         7.7       146.0  NA_Sales   
8         21.81          61.0          45.0         6.3       106.0  NA_Sales   
9         21.79          80.0          33.0         7.4        52.0  NA_Sales   

        lowest  Platform_2600  Platform_3DO  Platform_3DS  ...  Developer_nan  \
0     JP_Sales            0

# Model 2: MLPClassifier

MLPClassifier from sklearn

Splitting Training, Validation, and Testing Sets

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X = df1.drop(['highest','lowest'], axis=1)
lo = df1['lowest']
hi = df1['highest']

# Split data into train, validation, and test sets
# for highest
X_train, X_test, hi_train, hi_test = train_test_split(X, hi, test_size=0.2, random_state=427)

#for lowest
_, _, lo_train, lo_test = train_test_split(X, lo, test_size=0.2, random_state=427)

scaler = MinMaxScaler(feature_range = (0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Implementing Model

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report

# for highest
mlp = MLPClassifier(random_state=427)
mlp.fit(X_train, hi_train)
hi_pred = mlp.predict(X_test)
accuracy = accuracy_score(hi_test, hi_pred)
report = classification_report(hi_test, hi_pred)
print('Highest Region')
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(hi_test, hi_pred))

#for lowest
mlp = MLPClassifier(random_state=427)
mlp.fit(X_train, lo_train)
lo_pred = mlp.predict(X_test)
accuracy = accuracy_score(lo_test, lo_pred)
report = classification_report(lo_test, lo_pred)
print('Lowest Region')
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(lo_test, lo_pred))

C:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Highest Region
Accuracy: 0.81
              precision    recall  f1-score   support

    EU_Sales       0.67      0.66      0.66       275
    JP_Sales       0.52      0.51      0.51        91
    NA_Sales       0.87      0.88      0.88       989
 Other_Sales       0.33      0.10      0.15        10

    accuracy                           0.81      1365
   macro avg       0.60      0.54      0.55      1365
weighted avg       0.80      0.81      0.81      1365

Lowest Region
Accuracy: 0.76
              precision    recall  f1-score   support

    EU_Sales       0.56      0.53      0.54       207
    JP_Sales       0.86      0.85      0.85       891
    NA_Sales       0.66      0.61      0.64       132
 Other_Sales       0.56      0.69      0.62       135

    accuracy                           0.76      1365
   macro avg       0.66      0.67      0.66      1365
weighted avg       0.76      0.76      0.76      1365



C:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Hyperparameter Tuning for MLPClassifier

In [12]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report


# Initialize MLPClassifier
lo_clf = MLPClassifier(random_state=427)

# Define the hyperparameter distributions
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (50, 100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000]
}

# Initialize RandomizedSearchCV
lo_random_search = RandomizedSearchCV(
    estimator=lo_clf,
    param_distributions=param_distributions,
    n_iter=8,
    cv=3,
    scoring='f1_weighted',
    verbose=2,
    n_jobs=-1,
    random_state=427
)

# Fit RandomizedSearchCV
lo_random_search.fit(X_train, lo_train)

# Get the best hyperparameters
best_lo_params = lo_random_search.best_params_
print("Best parameters found: ", best_lo_params)

# Initialize a new model with the best parameters
best_lo_clf = MLPClassifier(**best_lo_params, random_state=427)

# Fit the model
best_lo_clf.fit(X_train, lo_train)

# Predict on the test data
lo_pred = best_lo_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy)
accuracy = accuracy_score(lo_test, lo_pred)
report = classification_report(lo_test, lo_pred)
print('Lowest Region')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

C:\Python312\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters found:  {'solver': 'adam', 'max_iter': 1000, 'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'relu'}
Accuracy: 0.7619047619047619
Lowest Region
Accuracy: 0.76
Classification Report:
              precision    recall  f1-score   support

    EU_Sales       0.55      0.54      0.54       207
    JP_Sales       0.86      0.85      0.85       891
    NA_Sales       0.66      0.64      0.65       132
 Other_Sales       0.57      0.67      0.61       135

    accuracy                           0.76      1365
   macro avg       0.66      0.67      0.67      1365
weighted avg       0.76      0.76      0.76      1365



In [13]:
# Initialize MLPClassifier
hi_clf = MLPClassifier(random_state=427)

# Define the hyperparameter distributions
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (50, 100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000]
}

# Initialize RandomizedSearchCV
hi_random_search = RandomizedSearchCV(
    estimator=lo_clf,
    param_distributions=param_distributions,
    n_iter=8,
    cv=3,
    scoring='f1_weighted',
    verbose=2,
    n_jobs=-1,
    random_state=427
)

# Fit RandomizedSearchCV
hi_random_search.fit(X_train, hi_train)

# Get the best hyperparameters
best_hi_params = hi_random_search.best_params_
print("Best parameters found: ", best_hi_params)

# Initialize a new model with the best parameters
best_hi_clf = MLPClassifier(**best_hi_params, random_state=427)

# Fit the model
best_hi_clf.fit(X_train, hi_train)

# Predict on the test data
hi_pred = best_hi_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy)
accuracy = accuracy_score(hi_test, hi_pred)
report = classification_report(hi_test, hi_pred)
print('Highest Region')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters found:  {'solver': 'adam', 'max_iter': 1000, 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50,), 'alpha': 0.01, 'activation': 'relu'}
Accuracy: 0.7611721611721611
Highest Region
Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

    EU_Sales       0.73      0.64      0.68       275
    JP_Sales       0.53      0.51      0.52        91
    NA_Sales       0.87      0.91      0.89       989
 Other_Sales       0.25      0.10      0.14        10

    accuracy                           0.82      1365
   macro avg       0.60      0.54      0.56      1365
weighted avg       0.81      0.82      0.81      1365



In [14]:
from joblib import Parallel, delayed
import joblib

#saving model
joblib.dump(best_lo_clf, 'Lowest_Region_MLPClassifier.pkl')
joblib.dump(best_hi_clf, 'Highest_Region_MLPClassifier.pkl')

['Highest_Region_MLPClassifier.pkl']